<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_9_tuning_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [9]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0     50      0  0:00:05  0:00:05 --:--:--    68
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [10]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [15]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,3,cracking sensations in my joints
1,5,fast and slow heartbeat
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [16]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-9.test.csv",header=None)
test.head()

,0,1
0,2,painful heels
1,4,Discoloration of skin [blotchy] on the neck
2,5,my heart rate jumped to 106
3,7,goiter
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [21]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [25]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.860209,4.203688,0.216518,03:37


In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('ft_fold9_first')

In [30]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold9_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.204255,3.397148,0.436658,01:12


In [31]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.136341,2.887502,0.491914,01:19


In [32]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.505248,2.509218,0.586253,03:28


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.174859,2.218198,0.623989,03:21
1,1.917946,2.092799,0.642857,03:18


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.742818,1.918911,0.677898,03:15
1,1.649936,1.799736,0.681941,03:23


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.454644,1.734795,0.706199,03:23
1,1.310719,1.616764,0.726415,03:20


In [36]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.210021,1.580763,0.721024,03:15
1,1.126417,1.498930,0.729111,03:23


In [37]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.115664,1.513805,0.726415,03:21
1,0.983362,1.456269,0.737197,03:16


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.962717,1.443959,0.743935,03:42
1,0.950593,1.394907,0.769542,03:32
2,0.828634,1.348434,0.770889,03:16
3,0.769586,1.335766,0.772237,03:12


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.774051,1.372432,0.766846,03:36
1,0.782117,1.332149,0.772237,03:29
2,0.724273,1.247909,0.788410,03:25
3,0.647023,1.222228,0.785714,03:28


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.676796,1.262070,0.788410,03:29
1,0.669966,1.290482,0.777628,03:19
2,0.647537,1.265004,0.797844,03:31
3,0.557002,1.224405,0.799191,03:28


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.578216,1.246508,0.797844,03:25
1,0.601714,1.262008,0.789757,03:15
2,0.608714,1.240207,0.788410,03:29
3,0.550329,1.199718,0.791105,03:31


In [0]:
# save the best model

learn.save_encoder('ask-9-tuning-cadec')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [44]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[861, 888, 444, 933, 8, 8, 8, 8, 8, 315, 16, 16, 20, 20, 974, 24, 27, 406, 27, 750, 31, 31, 31, 289, 37, 37, 37, 37, 37, 491, 289, 289, 289, 37, 38, 515, 1025, 49, 49, 49, 100, 840, 53, 53, 183, 60, 1035, 866, 816, 801, 71, 73, 76, 76, 343, 79, 265, 265, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 265, 211, 289, 289, 289, 289, 655, 717, 289, 80, 80, 491, 321, 85, 660, 660, 89, 89, 866, 815, 815, 92, 453, 453, 453, 453, 94, 94, 94, 94, 122, 421, 761, 725, 234, 98, 100, 100, 100, 100, 100, 288, 660, 660, 660, 108, 111, 111, 462, 111, 117, 117, 117, 117, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 127, 129, 343, 1025, 555, 268, 268, 137, 342, 499, 447, 449, 151, 151, 158, 160, 160, 321, 350, 350, 350, 350, 350, 350, 723, 172, 289, 289, 265, 289, 172, 172, 172, 173, 816, 624, 165, 5, 416, 182, 182, 289, 182, 183, 183, 183, 758, 758, 758, 184, 187, 187, 689, 188, 190, 190, 190,

In [45]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

565
0.6524249422632794
